In [4]:

import weaviate 
import pandas as pd
import numpy as np
import json
from weaviate.classes.config import Property, DataType,Configure
from weaviate.util import get_valid_uuid, generate_uuid5
from uuid import uuid4
import requests

In [2]:
air_bnb_data = pd.read_csv("AB_NYC_2019.csv")
air_bnb_data.dropna(inplace=True)
air_bnb_data.isnull().sum()
print(air_bnb_data.info())
data_list = air_bnb_data.to_dict('records')
data_list[0]


<class 'pandas.core.frame.DataFrame'>
Index: 38821 entries, 0 to 48852
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              38821 non-null  int64  
 1   name                            38821 non-null  object 
 2   host_id                         38821 non-null  int64  
 3   host_name                       38821 non-null  object 
 4   neighbourhood_group             38821 non-null  object 
 5   neighbourhood                   38821 non-null  object 
 6   latitude                        38821 non-null  float64
 7   longitude                       38821 non-null  float64
 8   room_type                       38821 non-null  object 
 9   price                           38821 non-null  int64  
 10  minimum_nights                  38821 non-null  int64  
 11  number_of_reviews               38821 non-null  int64  
 12  last_review                     38821

{'id': 2539,
 'name': 'Clean & quiet apt home by the park',
 'host_id': 2787,
 'host_name': 'John',
 'neighbourhood_group': 'Brooklyn',
 'neighbourhood': 'Kensington',
 'latitude': 40.64749,
 'longitude': -73.97237,
 'room_type': 'Private room',
 'price': 149,
 'minimum_nights': 1,
 'number_of_reviews': 9,
 'last_review': '2018-10-19',
 'reviews_per_month': 0.21,
 'calculated_host_listings_count': 6,
 'availability_365': 365}

In [3]:
client = weaviate.connect_to_local()
print('client is connected:',client.is_connected())
print(client.get_meta())

client is connected: True
{'hostname': 'http://[::]:8080', 'modules': {'generative-ollama': {'documentationHref': 'https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion', 'name': 'Generative Search - Ollama'}, 'text-spellcheck': {'model': {'name': 'pyspellchecker'}}, 'text2vec-ollama': {'documentationHref': 'https://github.com/ollama/ollama/blob/main/docs/api.md#generate-embeddings', 'name': 'Ollama Module'}}, 'version': '1.27.0'}


In [34]:
client.collections.delete(name="AirBnbListings")

air_bnb_listings = client.collections.create(
    name="AirBnbListings",
    description="A collection of AirBnb listings",
    vectorizer_config=Configure.Vectorizer.text2vec_ollama(model="llama3.2:latest", api_endpoint='http://host.docker.internal:11434'),
    generative_config=weaviate.classes.config.Configure.Generative.ollama(api_endpoint="http://host.docker.internal:11434", model = "llama3.2:latest"),
    properties=[
        # Property(name = 'name', data_type = DataType.TEXT),
        Property(name = 'name', data_type = DataType.TEXT),
        Property(name = 'host_id', data_type = DataType.TEXT, skip_vectorization=True),
        Property(name = 'host_name', data_type = DataType.TEXT,skip_vectorization=True),
        Property(name = 'neighbourhood_group', data_type = DataType.TEXT,skip_vectorization=True),
        Property(name = 'neighbourhood', data_type = DataType.TEXT,skip_vectorization=True),
        Property(name = 'latitude', data_type = DataType.NUMBER),
        Property(name = 'longitude', data_type = DataType.NUMBER),
        Property(name = 'room_type', data_type = DataType.TEXT),
        Property(name = 'price', data_type = DataType.NUMBER),
        Property(name = 'minimum_nights', data_type = DataType.NUMBER),
        Property(name = 'number_of_reviews', data_type = DataType.NUMBER),
        Property(name = 'last_review', data_type = DataType.TEXT),
        Property(name = 'reviews_per_month', data_type = DataType.NUMBER),
        Property(name = 'calculated_host_listings_count', data_type = DataType.NUMBER),
        Property(name = 'availability_365', data_type = DataType.NUMBER,skip_vectorization=True),
    ]
)

print(client.collections.list_all().keys())
# client.close()


dict_keys(['AirBnbListings', 'BookRecommendation', 'Ads'])


In [40]:
air_bnb = client.collections.get(name="AirBnbListings")
with air_bnb.batch.dynamic() as batch:
    for data in data_list:
        id = get_valid_uuid(uuid4()) 
        batch.add_object(uuid=id, properties=data)
        print(batch)


In [42]:
air_bnb = client.collections.get(name="AirBnbListings")

for item in air_bnb.iterator():  # limit can be adjusted as needed
    print("UUID:", item.uuid)
    print("Properties:", item.properties)
    print()

In [29]:
i =0
for data in data_list:
    i+=1
    data_properties = {}
    data_properties['name'] = data['name']
    data_properties['host_name'] = data['host_name']
    data_properties['neighbourhood_group'] = data['neighbourhood_group']
    data_properties['neighbourhood'] = data['neighbourhood']
    data_properties['latitude'] = data['latitude']
    data_properties['longitude'] = data['longitude']
    data_properties['room_type'] = data['room_type']
    data_properties['price'] = data['price']
    data_properties['minimum_nights'] = data['minimum_nights']
    data_properties['number_of_reviews'] = data['number_of_reviews']
    data_properties['last_review'] = data['last_review']
    data_properties['reviews_per_month'] = data['reviews_per_month']
    data_properties['calculated_host_listings_count'] = data['calculated_host_listings_count']
    data_properties['availability_365'] = data['availability_365']
    air_bnb_data.add
    row = air_bnb_listings.data.insert(uuid=id, properties=data_properties)
    print('Data inserted:',i)
    if i==1:
        break

    

Data inserted: 1


In [43]:
print("Total items in AirBnbListings collection:", sum(1 for _ in air_bnb.iterator()))


Total items in AirBnbListings collection: 0


In [39]:
air_bnb_listings = client.collections.get(name="AirBnbListings")
for item in air_bnb_listings.iterator(include_vector=True):
    print(item.uuid, item.properties)

In [ ]:
ad_schema = client.collections.create(
    name = 'ads',
    description = 'A collection of ads',
    properties = [
        Property(name = "content",data_type = DataType.TEXT)
    ]
)

In [56]:
#insert a new property to the airbnb collection
listing_hasAd_cref = {
    "dataType": ["ads"],
    "description": "The ad for an AirBnb Listing",
    "name": "hasAd"
}
air_bnb_listings.config.add_property(Property(listing_hasAd_cref))

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [57]:
generatePrompt = """
Please write a description for the following AirBnb Listing in english: 
Name: {name}
Neighbourhood: {neighbourhood}
Neighbourhood Group: {neighbourhood_group}
Latitude: {latitude}
Longitude: {longitude}
Room Type: {room_type}
Price: {price}
Minimum Nights: {minimum_nights}
Number of Reviews: {number_of_reviews}
Last Review: {last_review}
Reviews per Month: {reviews_per_month}
Calculated Host Listings Count: {calculated_host_listings_count}
Availability_365: {availability_365}

Please do not make up any information about the property in your description.
"""


generate_properties = ["name","neighbourhood","neighbourhood_group",
                       "latitude","longitude","room_type",
                       "price","minimum_nights","number_of_reviews",
                       "last_review","reviews_per_month",
                       "calculated_host_listings_count","availability_365"]

In [16]:
# descriptions = client.collections.get(name="AirBnbListings").query.hybrid(query=)



In [1]:
import json

request = requests.post("http://localhost:11434/api/generate", json={'model': 'llama3.2:latest','prompt' : 'what is rag in ai filed'})
res = request.text
full_response = ''
for line in res.strip().splitlines():
    obj = json.loads(line)
    full_response += obj["response"]

print(full_response)